# Baulking Functions - I

### This notebook will illustrate a simple deterministic example, designed to show the difference between baulking and queueing capacities

In [1]:
import ciw

In [2]:
def my_baulking_function(n):
    if n < 3:
        return 0.0
    return 1.0

In [3]:
params_dict = {
    'Arrival_distributions': [['Deterministic', 5.0], ['Deterministic', 23.0]],
    'Service_distributions': [['Deterministic', 21.0], ['Deterministic', 1.5]],
    'Transition_matrices': [[0.0, 0.0], [1.0, 0.0]],
    'Number_of_servers': [1, 1],
    'Baulking_functions': [my_baulking_function, None]
}

### Now if we run this to time t=48, we will get the following chain of events:

| Time | Queue Node 1 | In Service Node 1 | Queue Node 2 | In Service Node 2 | Customer Finished a Service | Baulked Customers |
|------|--------------|-------------------|--------------|-------------------|-----------------------------|-------------------|
| 0    |              |                   |              |                   |                             |                   |
| 5    |              | 1                 |              |                   |                             |                   |
| 10   | 2            | 1                 |              |                   |                             |                   |
| 15   | 3, 2         | 1                 |              |                   |                             |                   |
| 20   | 3, 2         | 1                 |              |                   |                             | 4                 |
| 23   | 3, 2         | 1                 |              | 5                 |                             | 4                 |
| 24.5 | 5, 3, 2      | 1                 |              |                   | 5                           | 4                 |
| 25   | 5, 3, 2      | 1                 |              |                   | 5                           | 4, 6              |
| 26   | 5, 3         | 2                 |              |                   | 5, 1                        | 4, 6              |
| 30   | 5, 3         | 2                 |              |                   | 5, 1                        | 4, 6, 7           |
| 35   | 5, 3         | 2                 |              |                   | 5, 1                        | 4, 6, 7, 8        |
| 40   | 5, 3         | 2                 |              |                   | 5, 1                        | 4, 6, 7, 8, 9     |
| 45   | 5, 3         | 2                 |              |                   | 5, 1                        | 4, 6, 7, 8, 9, 10 |
| 46   | 5, 3         | 2                 |              | 11                | 5, 1                        | 4, 6, 7, 8, 9, 10 |
| 47   | 5            | 3                 |              | 11                | 5, 1, 2                     | 4, 6, 7, 8, 9, 10 |
| 47.5 | 11, 5        | 3                 |              |                   | 5, 1, 2, 11                 | 4, 6, 7, 8, 9, 10 |
| 48   |              |                   |              |                   |                             |                   |

### We would therefore expect the following results:

In [4]:
expected_baulking_dictionary = {1:{0:[20.0, 25.0, 30.0, 35.0, 40.0, 45.0]}, 2:{0:[]}}
expected_ids_of_completed_customers = set([5, 1, 2, 11])
expected_waits_of_completed_customers = set([0.0, 0.0, 0.0, 16])
expected_arrival_dates_of_completed_customers = set([5.0, 10.0, 23.0, 46.0])
expected_service_start_dates_of_completed_customers = set([5.0, 23.0, 26.0, 46.0])
expected_service_end_dates_of_completed_customers = set([24.5, 26.0, 47.0, 47.5])

### Let's check:

In [5]:
N = ciw.create_network(params_dict)
Q = ciw.Simulation(N)

In [6]:
Q.simulate_until_max_time(48)
recs = Q.get_all_records()

In [7]:
Q.baulked_dict == expected_baulking_dictionary

True

In [8]:
set([r.id_number for r in recs]) == expected_ids_of_completed_customers

True

In [9]:
set([r.waiting_time for r in recs]) == expected_waits_of_completed_customers

True

In [10]:
set([r.arrival_date for r in recs]) == expected_arrival_dates_of_completed_customers

True

In [11]:
set([r.service_start_date for r in recs]) == expected_service_start_dates_of_completed_customers

True

In [12]:
set([r.service_end_date for r in recs]) == expected_service_end_dates_of_completed_customers

True